In [21]:
import pandas as pd
import numpy as np

In [22]:
vacancy_df = pd.read_csv('../Train_2.csv', encoding= 'ansi')

In [23]:
print('데이터 세트의 Shape:', vacancy_df.shape)

데이터 세트의 Shape: (6025, 69)


In [24]:
vacancy_df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X60,X61,X62,X63,X64,X65,X66,X67,X68,Y
0,173660,96866,76793,6356,52613,39480,32053,28652,14507,1314,...,2.651221e+09,1.261880e+09,2.020032e+09,1.600597e+09,141199,67904,118723,84751,110,50.2
1,209238,107003,102237,11228,45589,47885,44694,40624,19212,2514,...,1.396338e+09,6.259459e+08,9.079470e+08,6.824956e+08,60820,29820,50400,31521,63,50.2
2,478882,243693,235189,24111,83630,112851,109472,97958,50859,3743,...,5.488857e+09,1.855414e+09,3.330348e+09,3.451869e+09,285521,107030,215844,161208,219,50.2
3,1078154,556415,521736,63220,353808,235924,168800,157170,99228,1577,...,4.821905e+10,2.646531e+10,3.380276e+10,3.800171e+10,2571027,1394990,1784205,2117802,712,50.2
4,79702,48450,31256,3792,32157,16778,13658,10164,3155,578,...,3.334280e+09,1.056871e+09,2.203635e+09,1.687361e+09,264219,81132,186809,141214,110,50.2


In [5]:
#vacancy_df['Y'] = np.log1p(vacancy_df['Y'])

In [25]:
from scipy.stats import skew

In [26]:
# object가 아닌 숫자형 피쳐의 컬럼 index 객체 추출.
features_index = vacancy_df.dtypes[vacancy_df.dtypes != 'object'].index

# house_df에 컬럼 index를 [ ]로 입력하면 해당하는 컬럼 데이터 셋 반환. 
# apply lambda로 skew( )호출 
skew_features = vacancy_df[features_index].apply(lambda x : skew(x))

# skew 정도가 1 이상인 컬럼들만 추출.
skew_features_top = skew_features[skew_features > 1]

In [27]:
vacancy_df[skew_features_top.index] = np.log1p(vacancy_df[skew_features_top.index])

In [28]:
# skew_features_top.index[:-1]

In [29]:
vacancy_df = pd.get_dummies(vacancy_df)

In [30]:
vacancy_df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X64,X65,X66,X67,X68,Y,X56_HH,X56_HL,X56_LH,X56_LL
0,12.064860,11.481094,11.248882,8.757312,10.870738,10.583575,10.375177,10.263013,9.582456,7.181592,...,11.857933,11.125865,11.684557,11.347485,4.709530,3.93574,0,0,0,1
1,12.251232,11.580621,11.535059,9.326255,10.727444,10.776578,10.707617,10.612139,9.863342,7.830028,...,11.015690,10.302968,10.827766,10.358441,4.158883,3.93574,1,0,0,0
2,13.079212,12.403669,12.368149,10.090465,11.334170,11.633833,11.603433,11.492304,10.836832,8.227910,...,12.562074,11.580874,12.282316,11.990457,5.393628,3.93574,0,0,0,1
3,13.890762,13.229271,13.164919,11.054392,12.776512,12.371269,12.036476,11.965090,11.505186,7.363914,...,14.759816,14.148399,14.394484,14.565890,6.569481,3.93574,0,0,1,0
4,11.286063,10.788308,10.349999,8.240913,10.378417,9.727883,9.522154,9.226706,8.057061,6.361302,...,12.484537,11.303845,12.137847,11.858039,4.709530,3.93574,0,0,1,0


In [31]:
vacancy_df.shape

(6025, 72)

In [32]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [33]:
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds ):
    # 지정된 n_folds값으로 KFold 생성.
    kf = KFold(n_splits=n_folds, shuffle=False, random_state=0)
    #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화 
    train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ))
    test_pred = np.zeros((X_test_n.shape[0],n_folds))
    print(model.__class__.__name__ , ' model 시작 ')
    
    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출 
        print('\t 폴드 세트: ',folder_counter,' 시작 ')
        X_tr = X_train_n[train_index] 
        y_tr = y_train_n[train_index] 
        X_te = X_train_n[valid_index]  
        
        #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행.
        model.fit(X_tr , y_tr)       
        #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장. 
        test_pred[:, folder_counter] = model.predict(X_test_n)
            
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성 
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    
    
    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred , test_pred_mean

In [34]:
y_target = vacancy_df['Y']
X_features = vacancy_df.drop('Y',axis= 1, inplace=False)

X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, 
                                                    test_size= 0.2, random_state= 0)

In [35]:
X_train.shape, y_train.shape

((4820, 71), (4820,))

In [36]:
# get_stacking_base_datasets( )은 넘파이 ndarray를 인자로 사용하므로 DataFrame을 넘파이로 변환. 
X_train_n = X_train.values
X_test_n = X_test.values
y_train_n = y_train.values

In [37]:
def get_best_params(model, params):
    grid_model = GridSearchCV(model, param_grid=params, 
                              scoring='neg_mean_squared_error', cv=5)
    grid_model.fit(X_features, y_target)
    rmse = np.sqrt(-1* grid_model.best_score_)
    print('{0} 5 CV 시 최적 평균 RMSE 값: {1}, 최적 alpha:{2}'.format(model.__class__.__name__,
                                        np.round(rmse, 4), grid_model.best_params_))
    return grid_model.best_estimator_

In [38]:
ridge_reg = Ridge()
lasso_reg = Lasso()

ridge_params = { 'alpha':[0.05, 0.1, 1, 5, 8, 10, 12, 15, 20] }
lasso_params = { 'alpha':[0.001, 0.005, 0.008, 0.05, 0.03, 0.1, 0.5, 1,5, 10] }
best_rige = get_best_params(ridge_reg, ridge_params)
best_lasso = get_best_params(lasso_reg, lasso_params)

Ridge 5 CV 시 최적 평균 RMSE 값: 0.346, 최적 alpha:{'alpha': 0.05}


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 15.649552987408185, tolerance: 0.09150097099201786
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 19.576017771192483, tolerance: 0.0920268777413446
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.070280518914558, tolerance: 0.09247540929098978
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Dualit

Lasso 5 CV 시 최적 평균 RMSE 값: 0.348, 최적 alpha:{'alpha': 0.001}


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.643682660706759, tolerance: 0.11583999864782654
  positive)


In [39]:
ridge_reg = Ridge(alpha= 0.05)
lasso_reg = Lasso(alpha= 0.001)

In [40]:
xgb_reg = XGBRegressor(n_estimators=1000, learning_rate=0.05, 
                       colsample_bytree=0.5, subsample=0.8)
lgbm_reg = LGBMRegressor(n_estimators=1000, learning_rate=0.05, 
                         num_leaves=4, subsample=0.6, 
                         colsample_bytree=0.4, reg_lambda=10, 
                         n_jobs=-1)

In [41]:
ridge_train, ridge_test = get_stacking_base_datasets(ridge_reg, X_train_n, y_train_n, X_test_n, 5)
lasso_train, lasso_test = get_stacking_base_datasets(lasso_reg, X_train_n, y_train_n, X_test_n, 5)
xgb_train, xgb_test = get_stacking_base_datasets(xgb_reg, X_train_n, y_train_n, X_test_n, 5)  
lgbm_train, lgbm_test = get_stacking_base_datasets(lgbm_reg, X_train_n, y_train_n, X_test_n, 5)

Ridge  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
Lasso  model 시작 
	 폴드 세트:  0  시작 


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.713624226811646, tolerance: 0.07484341709529432
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.191808954783653, tolerance: 0.07619577753867127
  positive)


	 폴드 세트:  1  시작 


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.941190760667155, tolerance: 0.07380256506520802
  positive)


	 폴드 세트:  2  시작 


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.219181201207164, tolerance: 0.07344504751733155
  positive)


	 폴드 세트:  3  시작 


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 16.101888419144245, tolerance: 0.07514580215953197
  positive)


	 폴드 세트:  4  시작 
XGBRegressor  model 시작 
	 폴드 세트:  0  시작 
[14:48:36] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  1  시작 
[14:48:45] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  2  시작 
[14:48:54] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  3  시작 
[14:49:04] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  4  시작 
[14:49:13] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
LGBMRegressor  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1 

In [42]:
# 개별 모델이 반환한 학습 및 테스트용 데이터 세트를 Stacking 형태로 결합.  
Stack_final_X_train = np.concatenate((ridge_train, lasso_train, 
                                      xgb_train, lgbm_train), axis=1)
Stack_final_X_test = np.concatenate((ridge_test, lasso_test, 
                                     xgb_test, lgbm_test), axis=1)

In [43]:
Stack_final_X_train.shape

(4820, 4)

In [44]:
# 최종 메타 모델은 라쏘 모델을 적용. 
meta_model_lasso = Lasso(alpha=0.0005)

#기반 모델의 예측값을 기반으로 새롭게 만들어진 학습 및 테스트용 데이터로 예측하고 RMSE 측정.
meta_model_lasso.fit(Stack_final_X_train, y_train)
final = meta_model_lasso.predict(Stack_final_X_test)
mse = mean_squared_error(y_test , final)
rmse = np.sqrt(mse)
print('스태킹 회귀 모델의 최종 RMSE 값은:', rmse)

스태킹 회귀 모델의 최종 RMSE 값은: 0.11906056880121886


In [45]:
meta_model_lasso.score(Stack_final_X_train, y_train)

0.9102820412981494

In [46]:
ridge_pred = ridge_reg.predict(X_test)
print('SCORE :', ridge_reg.score(X_test, y_test))
mse_ridge = mean_squared_error(y_test, ridge_pred)
rmse_ridge = np.sqrt(mse_ridge)
print('RMSE :', rmse_ridge)

SCORE : 0.3987949897361227
RMSE : 0.3347742470489127


In [47]:
lasso_pred = lasso_reg.predict(X_test)
print('SCORE :', lasso_reg.score(X_test, y_test))
mse_lasso = mean_squared_error(y_test, lasso_pred)
rmse_lasso = np.sqrt(mse_lasso)
print('RMSE :', rmse_lasso)

SCORE : 0.3586684436508699
RMSE : 0.3457658131667755


In [48]:
# xgb_reg.fit(X_train, y_train)

In [29]:
#xgb_pred = lgbm_reg.predict(X_test)

In [30]:
lgbm_pred = lgbm_reg.predict(X_test)
print('SCORE :', lgbm_reg.score(X_test, y_test))
mse_lgbm = mean_squared_error(y_test, lgbm_pred)
rmse_lgbm = np.sqrt(mse_lgbm)
print('RMSE :', rmse_lgbm)

SCORE : 0.7726052626159521
RMSE : 0.20416003113148645


In [31]:
#xgb_pred = xgb_reg.predict(X_test)
#print('SCORE :', xgb_reg.score(X_test, y_test))
#mse_xgb = mean_squared_error(y_test, xgb_pred)
#rmse_xgb = np.sqrt(mse_xgb)
#print('RMSE :', rmse_xgb)

# SCORE : 0.8889663977998432
# RMSE : 0.14266181115946364

In [32]:
lasso_reg.score(X_test, y_test)

0.37668232396923695

In [49]:
f = pd.DataFrame(final)
ff = np.expm1(f)
y = pd.DataFrame(y_test)
yy = np.expm1(y)

result = pd.DataFrame()
result['Y_pred'] = ff.values.flatten()
result['Y_real'] = yy.values.flatten()
result['Mean'] = result['Y_real'] - result['Y_pred']
result['Mean'] = result['Mean'].abs()
result.describe()

,Y_pred,Y_real,Mean
count,1205.000000,1205.000000,1205.000000
mean,46.955745,47.362739,4.225406
std,32.076406,33.512927,9.122188
min,16.467907,20.900000,0.001197
25%,31.797820,29.800000,1.247644
50%,42.480656,43.200000,2.547700
75%,50.875721,53.800000,5.092822
max,351.934751,295.100000,210.972985


In [50]:
result.head()

,Y_pred,Y_real,Mean
0,74.595706,69.1,5.495706
1,32.626440,24.8,7.826440
2,39.874588,38.7,1.174588
3,44.529787,44.4,0.129787
4,49.900894,69.0,19.099106


In [59]:
X_test_org = pd.read_csv('../임대료용/Test.csv', encoding= 'ansi')

In [60]:
skew_features_top_X = skew_features_top.index[:-1]

In [61]:
X_test_org[skew_features_top_X] = np.log1p(X_test_org[skew_features_top_X])
X_test_org.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X59,X60,X61,X62,X63,X64,X65,X66,X67,X68
0,12.064860,11.481094,11.248882,8.757312,10.870738,10.583575,10.375177,10.263013,9.582456,7.181592,...,12.250587,21.698286,20.955869,21.426379,21.193643,11.857933,11.125865,11.684557,11.347485,4.709530
1,12.251232,11.580621,11.535059,9.326255,10.727444,10.776578,10.707617,10.612139,9.863342,7.830028,...,11.414662,21.057119,20.254775,20.626697,20.341267,11.015690,10.302968,10.827766,10.358441,4.158883
2,13.079212,12.403669,12.368149,10.090465,11.334170,11.633833,11.603433,11.492304,10.836832,8.227910,...,12.880424,22.425986,21.341374,21.926343,21.962182,12.562074,11.580874,12.282316,11.990457,5.393628
3,13.890762,13.229271,13.164919,11.054392,12.776512,12.371269,12.036476,11.965090,11.505186,7.363914,...,15.193273,24.599020,23.999101,24.243808,24.360897,14.759816,14.148399,14.394484,14.565890,6.569481
4,11.286063,10.788308,10.349999,8.240913,10.378417,9.727883,9.522154,9.226706,8.057061,6.361302,...,12.752319,21.927523,20.778579,21.513374,21.246432,12.484537,11.303845,12.137847,11.858039,4.709530


In [63]:
X_test_ = pd.get_dummies(X_test_org)
X_test_.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X63,X64,X65,X66,X67,X68,X56_HH,X56_HL,X56_LH,X56_LL
0,12.064860,11.481094,11.248882,8.757312,10.870738,10.583575,10.375177,10.263013,9.582456,7.181592,...,21.193643,11.857933,11.125865,11.684557,11.347485,4.709530,0,0,0,1
1,12.251232,11.580621,11.535059,9.326255,10.727444,10.776578,10.707617,10.612139,9.863342,7.830028,...,20.341267,11.015690,10.302968,10.827766,10.358441,4.158883,1,0,0,0
2,13.079212,12.403669,12.368149,10.090465,11.334170,11.633833,11.603433,11.492304,10.836832,8.227910,...,21.962182,12.562074,11.580874,12.282316,11.990457,5.393628,0,0,0,1
3,13.890762,13.229271,13.164919,11.054392,12.776512,12.371269,12.036476,11.965090,11.505186,7.363914,...,24.360897,14.759816,14.148399,14.394484,14.565890,6.569481,0,0,1,0
4,11.286063,10.788308,10.349999,8.240913,10.378417,9.727883,9.522154,9.226706,8.057061,6.361302,...,21.246432,12.484537,11.303845,12.137847,11.858039,4.709530,0,0,1,0


In [64]:
X_test_n = X_test_.values

In [70]:
ridge_train_ = ridge_reg.predict(X_test_n).reshape(-1, 1)
lasso_train_ = lasso_reg.predict(X_test_n).reshape(-1, 1)
lgbm_train_ = lgbm_reg.predict(X_test_n).reshape(-1, 1)
xgb_train_ = xgb_reg.predict(X_test_n).reshape(-1, 1)

In [71]:
Stack_final_X_test = np.concatenate((ridge_train_, lasso_train_, 
                                     xgb_train_, lgbm_train_), axis=1)

In [72]:
final_ = meta_model_lasso.predict(Stack_final_X_test)

In [73]:
X_final = np.expm1(final_)

In [74]:
X_final_ = pd.DataFrame(X_final)

In [77]:
X_final_.to_csv('Result.csv', encoding= 'ansi')